# Growing std::vector

`std::vector<T>` stores its contents contiguously in memory. This makes access to its elements simple and cache friendly, usually leading to nice performance. Nevertheless, it is a flexible dynamically sized container managing resources on the heap. In the class, we discussed how the growth of the vector entails occasional relocation of all the elements to a new location. This exercise demonstrates it.

In [ ]:
#pragma cling add_include_path("/p/project/training2444/local/include")
#include <Vbose.hh>
#include <vector>

Let's start by creating an object of a type `Vbose`, whose definition is in the header `Vbose.hh`, distributed with the course material. The constructors, destructors and other special member functions of `Vbose` output messages when they are used. Observe:

In [ ]:
Vbose a{"2024"};

In [ ]:
{
    Vbose b{a};
} // b goes out of scope here!

Now, let's examine what happens to the elements when we grow a vector by creating a `std::vector<Vbose>`and inserting elements into it.

In [ ]:
std::vector<Vbose> v;
v.reserve(4UL);

In [ ]:
v.push_back(a);

In [ ]:
v.push_back(a);

In [ ]:
v.push_back(a);

In [ ]:
v.push_back(a);

In [ ]:
v.push_back(a);

Does the behaviour correspond to your expectations? When the vector grows beyond the previously reserved capacity, it allocates a new block of memory, constructs the new object at its expected location in the block and moves all previous elements to the new block.

It should be immediately clear that if we had stored a pointer or an iterator or a reference to an element of the vector, and continued grow the vector, at some point, those pointers, references or iterators could all be potentially invalidated: the objects they were pointing to could be moved to a new block of memory as the vector grows.

In [ ]:
{
    std::vector<Vbose> v2;
    v2.reserve(2);
    v2.push_back(a);
    auto* ptr = &v2[0]; // Bad idea!
    auto& ref = v2.front(); // Bad idea!
    auto it = v2.begin(); // Bad idea!
    std::cout << "Through pointer: " << ptr->value() << "\n";
    std::cout << "Through reference: " << ref.value() << "\n";
    std::cout << "Through iterator: " << it->value() << "\n";
    v2.push_back(a);
    std::cout << "Through pointer: " << ptr->value() << "\n";
    std::cout << "Through reference: " << ref.value() << "\n";
    std::cout << "Through iterator: " << it->value() << "\n";
    v2.push_back(a); // Entering crazy town...
 
    // Uncommenting any of the following lines will likely
    // result in a SEGFAULT now in a normal terminal based run.
    // Since we are in a Jupyter kernel, it will crash the
    // kernel. Do try it! When it creashes, use the Restart Kernel
    // button and re-run all the cells up to this point.
    
    // std::cout << "Through pointer: " << ptr->value() << "\n";
    // std::cout << "Through reference: " << ref.value() << "\n";
    // std::cout << "Through iterator: " << it->value() << "\n";
}

A safety recommendation: __Do not store element references to elements inside a vector which may grow__


Now, another aspect of vectors since C++11 which often gets misused: `emplace_back`. To demonstrate, let's first fix a new capacity of our vector, so that we can start to anticipate the copies and moves...

In [ ]:
v.reserve(v.size() + 20);

`emplace_back()` is there to enable _in place_ construction of new objects in a container. However, it often gets used like this:

In [ ]:
v.emplace_back(a);

or this...

In [ ]:
v.emplace_back(Vbose{"Frog"});

or this...

In [ ]:
v.emplace_back<Vbose>({"Frog"});

As you can see, the above ways of using `emplace_back` are even worse than `push_back`, since we first create an object using the parameters given to `emplace_back`, move it to the data block, and destruct the moved from object. `push_back` simply copy constructs the object in the final location. `emplace_back` calls the constructor of the value type using the parameters given to it. The template arguments specify what is being entered inside the parentheses of `emplace_back` rather than the value type of the container. The compiler already knows that value type, and it would be a silly syntax to require that we specify that as above. The right way to use `emplace_back` is this: 

In [ ]:
v.emplace_back("Cat");

In this case, we directly created a new object inside the vector, using new data. Importantly, a `Vbose` object with the string "Cat" didn't exist, unlike the `push_back` examples above, where the source of the copy construction existed in our object `a`. To illustrate this point, let's consider how we would use `push_back` to insert a new object with new data, and contrast that to the same using `emplace_back`:

In [ ]:
v.push_back(Vbose{"rhino"});

In [ ]:
v.push_back({"ram"});

In [ ]:
v.emplace_back("rabbit");

Notice the difference? No construction-move-destruction, but rather a single call to the relevant constructor. Since the standard library programmers did not know what parameters our `Vbose` objects will need in the constructor, how did they write `emplace_back`?

Functions like `emplace_back` are written using variadic templates. When we call `v.emplace_back(x, y, z)`, inside the implementation of `std::vector`, there is a call to a constructor `value_type (x, y, z)`. The inputs are _forwarded_ to the constructor as they are. But that's another topic!